In [1]:
import pandas as pd
import numpy as np
from scipy.stats import norm
import yfinance as yf
import plotly.express as px
from sklearn.metrics import mean_absolute_error

In [3]:
## Monte Carlo

# dataset = pd.read_csv("acoes.csv")
# figura = px.line(title = 'Histórico do preço das ações')
# for i in dataset.columns[1:]:about:blank#blocked
#   figura.add_scatter(x = dataset['Date'], y = dataset[i], name = i)
# figura.show()
empresa = input("Escreva o nome da empresa")
simulacoes = int(input("Qntd Simulações"))
dias_frente = int(input("Quantos dias a frente?"))
dataset = pd.read_csv('excel/acoes.csv')
dataset = pd.DataFrame(dataset[f"{empresa}"])
dataset_normalizado = dataset.copy()
for i in dataset:
  dataset_normalizado[i] = dataset[i] / dataset[i][0]
dataset_taxa_retorno = np.log(1 + dataset_normalizado.ffill().pct_change())
dataset_taxa_retorno.fillna(0, inplace=True)
# dataset_taxa_retorno

media = dataset_taxa_retorno.mean() 
variancia = dataset_taxa_retorno.var()
drift = media - (0.5 * variancia)
# drift

desvio_padrao = dataset_taxa_retorno.std()
desvio_padrao
Z = norm.ppf(np.random.rand(dias_frente, simulacoes))

retornos_diarios = np.exp(drift.values + desvio_padrao.values * Z)
previsoes = np.zeros_like(retornos_diarios)
previsoes[0] = dataset.iloc[-1] # escolhe o ultimo valor

for dia in range(1, dias_frente):
  #print(dia)
  previsoes[dia] = previsoes[dia - 1] * retornos_diarios[dia]

figura = px.line(title = 'Previsões do preço das ações - simulações')
for i in range(len(previsoes.T)):
  figura.add_scatter(y = previsoes.T[i], name = i)

#figura.show()  

dataset_empresa = yf.download(f"{empresa}", start='2023-10-12')['Close']
dataset_empresa.to_csv('empresa.csv')
dataset_empresa = pd.read_csv('empresa.csv')
dataset_empresa
simulacao1 = previsoes.T[0][0:len(dataset_empresa)]
print(len(simulacao1), len(dataset_empresa))
np.sum(abs(simulacao1 - dataset_empresa['Close'])) / len(simulacao1)

erros = []

for i in range(len(previsoes.T)):
  simulacao = previsoes.T[i][0:len(dataset_empresa)]
  erros.append(mean_absolute_error(dataset_empresa['Close'], simulacao))

result = [(num, idx) for idx, num in enumerate(erros) if num < 1]
print(result)  



[*********************100%%**********************]  1 of 1 completed
5 29


ValueError: operands could not be broadcast together with shapes (5,) (29,) 

In [ ]:
figura = px.line(title = 'Previsões do preço das ações - simulação')
figura.add_scatter(y = dataset_empresa['Close'], name = f'Valor {empresa}')
figura.add_scatter(y = previsoes.T[837], name = 'Melhor simulação')
